# Multi-Modal Advertisement Analytics using Snowflake Cortex AI SQL
Effective advertisement campaign analysis requires understanding both structured data (impressions, clicks, metadata) and unstructured content (images, visual elements). In this notebook, you'll use Cortex AI SQL to systematically analyze advertisement images, extract insights, and optimize campaign performance through intelligent classification and content generation.

### Context
This demonstrates how modern marketing analytics teams can leverage Snowflake's Cortex AI SQL to analyze multi-modal advertising data. The system processes advertisement images from campaigns, analyzes visual content using AI functions, classifies ad types, and generates insights to optimize future campaigns.

In this notebook, we will leverage multiple AI SQL functions to build an intelligent advertisement analytics workflow that:
- Filters advertisements based on specific visual criteria using **AI_FILTER**
- Classifies advertisement types automatically with **AI_CLASSIFY**
- Generates creative taglines and content using **AI_COMPLETE**
- Aggregates customer feedback and sentiment analysis with **AI_AGG**
- Processes image files using the new **FILE data type**

The primary business value is creating a more intelligent and scalable approach to advertisement campaign analysis that enables marketing teams to understand visual content performance, optimize creative assets, and make data-driven decisions about future campaigns.

##

## Step 1: Set up your environment and data

Let's begin by running the query below. It sets the correct role and warehouse for this session and creates a new schema. It also creates the necessary stage for image storage and sample tables for our advertisement analytics workflow.

**Note**: Upload your advertisement images to the **ads_image_stage** stage before proceeding with the rest of the notebook.

In [ ]:
import snowbooks_extras

In [ ]:
USE ROLE SNOWFLAKE_LEARNING_ROLE;

-- Use the existing database, schema and warehouse
USE DATABASE SNOWFLAKE_LEARNING_DB;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;

SET schema_name = CONCAT(current_user(), '_ADS_IMAGE_ANALYTICS');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($schema_name);
USE SCHEMA IDENTIFIER($schema_name);

/*--
• Stage creation for image file storage
--*/

CREATE OR REPLACE STAGE ads_image_stage 
  DIRECTORY = (ENABLE = true) 
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
  COMMENT = 'Stage for advertisement image files';

/*--
• Sample data tables creation
--*/

CREATE OR REPLACE TABLE synthetic_impressions (
    impression_id NUMBER,
    customer_id STRING,
    event_timestamp TIMESTAMP_NTZ,
    image_file_name STRING,
    image_type STRING, 
    was_clicked BOOLEAN,
    ip_address STRING,
    latitude FLOAT,
    longitude FLOAT,
    city STRING,
    state STRING
);

-- Setup completion note
SELECT 'Advertisement analytics setup is complete' AS note;

In [ ]:
----------------------------------------------------------------------------------------------------
--- Create images table from uploaded files in the stage
----------------------------------------------------------------------------------------------------

CREATE OR REPLACE TABLE images AS
SELECT
    TO_FILE(file_url) AS img_file,
    file_url,
    relative_path,
    last_modified
FROM
    directory(@ads_image_stage)
WHERE
    relative_path LIKE '%.png'
    AND relative_path NOT LIKE '%instructions%';

## Step 2: Create images table from uploaded files

Now we'll create a table that references the uploaded image files using Snowflake's **FILE data type**. This allows us to work with the images directly in SQL queries while maintaining the file references. The query below processes all PNG files from the stage, excluding any instruction files.

In [ ]:
SELECT
    img_file,
    file_url,
    last_modified
FROM ai_sql_demo_db.impressions.images;

In [ ]:
-- Review the table structure and data types
DESCRIBE TABLE images;

## Step 3: Generate creative taglines using AI_COMPLETE

Using the **AI_COMPLETE** function, we can analyze advertisement images and create fresh, compelling taglines for our campaigns. This demonstrates how AI can extract visual information and transform it into marketing copy that resonates with target audiences.

In [ ]:
SELECT
    relative_path AS image_name,
    AI_COMPLETE(
        'pixtral-large', 
        PROMPT(
            'Extract the text from this image and create an advertising tagline. Just return the tagline itself: {0}', 
            img_file
        )
    ) AS generated_tagline,
    file_url
FROM images
ORDER BY relative_path;

## Step 4: Filter advertisements with AI_FILTER

Historical campaign data suggests that including coffee-related equipment in advertisements significantly boosts click-through rates. Let's use **AI_FILTER** to identify and isolate advertisements that contain coffee makers, espresso machines, or related brewing equipment for our targeted campaign analysis.

In [ ]:
SELECT 
    relative_path AS image_name,
    img_file, 
    file_url, 
    last_modified 
FROM images
WHERE AI_FILTER(
    PROMPT(
        'Does this image {0} contain an espresso machine, coffee pot, french press, or other coffee brewing equipment?', 
        img_file
    )
)
ORDER BY relative_path;

## Step 5: Classify advertisement types with AI_CLASSIFY

Now let's use **AI_CLASSIFY** to automatically categorize our advertisements into different types. This classification helps us understand our creative portfolio composition and optimize our campaign mix based on performance data. We'll classify ads into categories like product shots, lifestyle imagery, promotional offers, and text-heavy advertisements.

In [ ]:
CREATE OR REPLACE TABLE images_classified AS
SELECT
    img_file,
    SPLIT_PART(relative_path, '/', -1) AS file_name,
    AI_CLASSIFY(
        PROMPT(
            'Please help me classify the type of advertisement within this coffee-related image {0}', 
            img_file
        ), 
        [
            'long text', 
            'product shot', 
            'lifestyle', 
            'promo offer'
        ]
    ) AS classification_result,
    PARSE_JSON(classification_result):labels[0]::TEXT AS ad_type,
    file_url, 
    relative_path,
    last_modified
FROM images;

-- Review the classified advertisements
SELECT 
    file_name,
    ad_type,
    file_url
FROM images_classified
ORDER BY ad_type, file_name;

## Step 6: Analyze customer feedback with AI_AGG

Using **AI_AGG** to aggregate and analyze customer feedback, we can effectively evaluate our advertisement campaign performance. This deeper understanding enables us to craft more targeted advertising and enhance customer engagement for future campaigns based on sentiment analysis and key insights.

In [ ]:
SELECT
    image_type,
    AI_AGG(
        'Image Type: ' || image_type || '\n\nFeedback: ' || impression_review, 
        'Score the sentiment of the feedback on a scale of 1 to 5 where 1 is very negative and 5 is very positive. Just write 1, 2, 3, 4, or 5. Return the average value only'
    ) AS sentiment_score,
    AI_AGG(
        'Image Type: ' || image_type || '\n\nFeedback: ' || impression_review, 
        'Analyze the feedback from these reviews and summarize the key insights in one concise sentence'
    ) AS feedback_summary 
FROM impressions_with_reviews
GROUP BY image_type
ORDER BY sentiment_score DESC;

## Key Takeaways

* **Multi-Modal Data Processing**: You can leverage Snowflake's new FILE data type to work directly with image files in SQL queries, enabling seamless analysis of visual content alongside structured data.
* **Intelligent Content Analysis**: AI-powered functions like `AI_FILTER`, `AI_CLASSIFY`, `AI_COMPLETE`, and `AI_AGG` enable comprehensive advertisement analysis from visual content extraction to sentiment analysis.
* **Automated Creative Generation**: The `AI_COMPLETE` function can generate marketing taglines and content recommendations based on visual analysis, scaling creative development processes.
* **Visual Content Classification**: Automated categorization of advertisement types helps marketing teams understand portfolio composition and optimize campaign strategies.
* **Sentiment-Driven Insights**: AI-powered aggregation of customer feedback provides actionable insights for improving advertisement effectiveness and customer engagement.

## Additional Resources

* [Documentation: Cortex AI SQL Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql)
* [Documentation: FILE Data Type](https://docs.snowflake.com/en/sql-reference/data-types-file)